# CLAPPING MUSIC

An algorithmic implementation of Steve Reich's [Clapping Music](https://www.youtube.com/watch?v=u55XIK_4buI).

<hr style="height:1px; color: grey;">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import Score, Note, MidiEvent, Seq, MidiFile, version, setmidiplayer, playfile, rhythm
from musx.midi.gm import HiWoodBlock, LowBongo, HandClap, Cowbell, Maracas
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print(f'version: {version}')

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

Define 'parameters' that set the initial conditions for the part generators. Experiment with the various values to see how it affects the composition.

In [ ]:
# Encoding of one measure of the essential pattern; 1 denotes a sounding note and 0 is a rest.
pattern = [1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0]

# Number of times the measures are repeated before moving to the next measure. 
# Reich repeats each measure 12 times but for our testing we start small!
repeat = 2

# the rate of 'clapping', in seconds. ('e' stands for eight-note, see documentation)
rate = rhythm("e", tempo=160)

# The midi drum map key numbers to use as 'claps'. (see musx.midi.gm documentation)
# Its easier to hear if you pick two distinct drum map sounds...experiment!
pitch1 = HiWoodBlock #Cowbell
pitch2 = LowBongo #Low Timbale #LowWoodBlock #HandClap #Maracas

# Loudness of the claps (0 - 1.0). Higher voice should probably be quieter than the lower.
amp1 = .7
amp2 = .9 
print(f"pattern: {pattern}\nrepeat: {repeat}\nrate: {rate}\npitch1: {pitch1}\npitch2: {pitch2}\namp1: {amp1}\namp2: {amp2}")

Define a part composer that can generate both parts for the piece.
The generator's parameters are:

* score - the musx Score object to fill.
* pattern - a list of 1's and 0's where 1 indicates an (audible) note and 0 represents a (silent) rest. 
* repeat - how many times each measure is repeated before moving to the next measure.
* rate - rhythmic speed of the notes and rests.
* pitch - the midi drum map key number to perform.
* amp - the loudness of the performance.
* rotate - if True then the pattern is rotated for each new measure, if False the pattern never changes.

In [ ]:
def clapper(score, pattern, repeat, rate, pitch, amp, rotate):
    assert repeat > 0, "repeat should be greater than zero"
    performance = []
    
    # build the part's performance given the pattern and repeat values
    for _ in range(len(pattern) + 1) :
        performance += pattern * repeat
        # if rotatating move the front element to the back.
        if rotate:
            pattern = pattern[1:] + [pattern[0]]

    # add notes with their parameterized charateristics to the score 
    for p in performance:
        if p == 1:  # notes are assigned to the midi drum channel (channel 9 in musx)
            n = Note(time=score.now, duration=rate, pitch=pitch, amplitude=amp, instrument=9)
            score.add(n)
        # no matter what wait rate amount of time before playing the next note.
        yield rate

print(f"clapper: {clapper}")

Create a Score object and give it an empty Seq (sequence) object to fill. Compose the score passing it two instances of `clapper()`, each with its specific parameters. Once the score is composed, its output sequence is added to a MidiFile and written to the hard drive. Assuming you have installed a terminal-based midi player the file will be auto-played:

In [ ]:
score = Score(out=Seq())
score.compose( [clapper(score, pattern, repeat, rate, pitch1, amp1, False),
                clapper(score, pattern, repeat, rate, pitch2, amp2, True)])
file = MidiFile("clapping.mid", [score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)